In [1]:
import getpass
import os
import bs4
from langchain import hub
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

os.environ["LANGSMITH_TRACING"] = "false"

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
embeddings = OllamaEmbeddings(model="deepseek-r1:70b")
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db/",
)

In [3]:
# Load and chunk contents of the blog
#loader = WebBaseLoader(
#    web_paths=("http://www.dengzhou.gov.cn/dzszj/zjxw/zwyw/webinfo/2024/11/1723956574402713.htm",),
#    bs_kwargs=dict(
#        parse_only=bs4.SoupStrainer(
#            class_=("post-content", "post-title", "post-header")
#        )
#    ),
#)
loader = PyMuPDFLoader(
    file_path="../library/MIT/36829799-MIT-A combination of structurally connected and multiple spacecraft interferometers.pdf"
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
loader = PyMuPDFLoader(
    file_path="../library/MIT/40170222-MIT-Comparative system trades between structurally connected and separated spacecraft interferometers for the terrestrial planet finder mission.pdf"
)
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits += text_splitter.split_documents(docs)

In [4]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [6]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOllama(
    model="deepseek-r1:70b",
    temperature=0.0,
    num_predict=-1
)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [7]:
response = graph.invoke({"question": "What are the limits of structurally connected multiple spacecraft interferometer?"})
print(response["answer"])

<think>
Okay, so I need to figure out what the limits of a structurally connected multiple spacecraft interferometer are based on the provided context. Let me go through this step by step.

First, looking at the context, there's mention of figures and tables related to parameters like thruster force, baseline distances, and constraints such as C2, C3, etc. It seems like these constraints are affecting the design or performance of the interferometer.

I see that Figure 5-18 talks about aspect ratio and areal density being limited by dry mass (C1), frequency (C2), and minimum gage (C5). So, these must be some of the key limiting factors. The context also mentions that at a 10 m baseline, only certain constraints like C2 are active, which probably means they impose specific limits on how the interferometer can operate.

There's another part about propellant mass rates and mission durations. It says that for missions longer than 367 years, the propellant mass fraction becomes a concern, bu

In [8]:
response = graph.invoke({"question": "What is your suggestion for an interferometer with 300-metre baseline?"})
print(response["answer"])

<think>
Okay, so I need to figure out what the suggestion is for an interferometer with a 300-meter baseline based on the provided context. Let me start by reading through the context carefully.

The context mentions several figures and tables related to different parameters of an interferometer. It talks about operational scenarios where both rotating and non-rotating interferometers are studied, but it doesn't specify which one is better for a 300-meter baseline. 

Looking at Figure 5-40, it shows average propellant mass rates versus baseline for mission lengths from one week to thirty years. The critical time curve in Figure 5-42 has a steep slope because the propellant mass rates are small. It also mentions that even though MSI's propellant rate is higher than SCI's, the absolute magnitude is so small that it would take a long time for the extra propellant needed for MSI to equal the extra structural mass of SCI.

Figure 5-41 includes a constraint (C8) for MSI where the propellant 

In [9]:
response = graph.invoke({"question": "What are the differences between a separated spacecraft interferometer and a structurally connected multiple spacecraft interferometer?"})
print(response["answer"])

<think>
Okay, so I need to figure out the differences between a separated spacecraft interferometer and a structurally connected multiple spacecraft interferometer based on the provided context. Let me start by reading through the context carefully.

Looking at the context, it seems to be from a technical document discussing various parameters related to space interferometers. There are mentions of figures like Figure 5-27, 5-28, etc., which probably contain data on things like thruster force, baseline lengths, stress, and propellant mass rates.

I notice that terms like "SCI" and "MSI" appear in the context. I'm guessing SCI stands for Structurally Connected Interferometer and MSI might be Multiple Spacecraft Interferometer or something similar. The figures compare parameters between these two systems over different baselines, which is the distance between the spacecraft.

In Figure 5-40, it shows average propellant mass rates versus baseline for both SCI and MSI across mission durati